In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv("/home/miguel/Desktop/ML_Notebooks_&_Projects/30_Dias/Dia_9/First_model/melb_data.csv")
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]
y = data.Price
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [6]:
X_train.head()

,Rooms,Distance,Landsize,BuildingArea,YearBuilt
12167,1,5.0,0.0,NaN,1940.0
6524,2,8.0,193.0,NaN,NaN
8413,3,12.6,555.0,NaN,NaN
2919,3,13.0,265.0,NaN,1995.0
6043,3,13.3,673.0,673.0,1970.0


# XGBoost

In [7]:
from xgboost import XGBRegressor
model = XGBRegressor()
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [8]:
# Evaluacion
from sklearn.metrics import mean_absolute_error
preds = model.predict(X_valid)
print("MAE: ", mean_absolute_error(y_valid, preds))

MAE:  236021.00835615335


# Tuning de XGBoost

In [9]:
# N_estimators = numero de ciclos de boosting
modelo_2 = XGBRegressor(n_estimators = 500)
modelo_2.fit(X_train, y_train)
preds = modelo_2.predict(X_valid)
print("MAE con modelo 2:\n", mean_absolute_error(preds, y_valid))

MAE con modelo 2:
 248417.43546345728


In [12]:
# Early stopping: cada cuanto el modelo checkea si esta empeorando para parar de entrenar
modelo = XGBRegressor(n_estimators = 500)
modelo.fit(X_train, y_train,
          early_stopping_rounds = 5,
          eval_set = [(X_valid, y_valid)],
          verbose = False)
preds = modelo.predict(X_valid)
print("MAE con modelo 2:\n", mean_absolute_error(preds, y_valid))

MAE con modelo 2:
 238594.2109892765


In [13]:
# Learning_rate: numero por el que multiplico cada modelo en el ensembling
modelo = XGBRegressor(n_estimators = 500, learning_rate = 0.05)
modelo.fit(X_train, y_train,
          early_stopping_rounds = 5,
          eval_set = [(X_valid, y_valid)],
          verbose = False)
preds = modelo.predict(X_valid)
print("MAE con modelo 3:\n", mean_absolute_error(preds, y_valid))

MAE con modelo 3:
 242422.7209131075


In [14]:
# n_jobs: para que el modelo use paralelismo, n_jobs = Numero de cores
modelo = XGBRegressor(n_estimators = 500, learning_rate = 0.05, n_jobs = 4)
modelo.fit(X_train, y_train,
          early_stopping_rounds = 5,
          eval_set = [(X_valid, y_valid)],
          verbose = False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [15]:
preds = modelo.predict(X_valid)
print("MAE con modelo 4:\n", mean_absolute_error(y_valid, preds))

MAE con modelo 4:
 242422.7209131075
